In [8]:
import nltk
import pandas as pd
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import  cosine_similarity
df_train = pd.read_excel("../dataset/ex_merged.xlsx")
#df_train = pd.read_csv("https://raw.githubusercontent.com/SNMHZ/Drug_Recommendation/master/dataset/lem_train.csv", parse_dates=["date"], infer_datetime_format=True)
#df_test = pd.read_csv("https://raw.githubusercontent.com/SNMHZ/Drug_Recommendation/master/dataset/lem_test.csv", parse_dates=["date"], infer_datetime_format=True)

print(type(df_train['review']))

#df_train['tokenize_review'] = word_tokenize(df_train['review'])

print(df_train.isnull().values.any())
#print(df_test.isnull().values.any())

print(len(df_train))
#print(len(df_test))

df_train = df_train.dropna(how='any')
#df_test = df_test.dropna(how='any')

print(df_train.isnull().values.any())
#print(df_test.isnull().values.any())

print(len(df_train))
#print(len(df_test))
print(df_train['review'][0])

#df_train['new'] = df_train['review'].str.cat(df_train['condition'], sep=' ')
#df_train['new'] = df_train['new'].str.lower()

<class 'pandas.core.series.Series'>
False
809
False
809
severe pain nausea prescribed medication still pain nausea tolerable severe pain nausea prescribed medication still pain nausea tolerable 


In [16]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas, nltk
from nltk.tokenize import RegexpTokenizer

#df_all = pd.concat([df_train, df_test])

def nltk_tokenizer(words):
  return RegexpTokenizer(r'\n+').tokenize(words.lower())
df_train['tokenized'] = [word_tokenize(sentence) for sentence in df_train['review']]
print(df_train.head())
doc_df = df_train[['condition2', 'tokenized']].values.tolist()
tagged_data = [TaggedDocument(words=_d, tags=[uid]) for uid, _d in doc_df]
max_epochs = 10

model = Doc2Vec(
    window=10,
    vector_size=150,
    alpha=0.025, 
    min_alpha=0.025,
    min_count=2,
    dm =1,
    negative = 5,
    seed = 9999, compute_loss = True)

model.build_vocab(tagged_data)

print(tagged_data[:5])

   Unnamed: 0               condition2  \
0           0      abdominaldistension   
1           1  abnormaluterinebleeding   
2           2                 abortion   
3           3    acetaminophenoverdose   
4           4         aciallipoatrophy   

                                              review  \
0  severe pain nausea prescribed medication still...   
1  severe bleeding large clot last week finally s...   
2  take medicine able find heartbeat bad pain exp...   
3  really early morning go take tylenol misread d...   
4  try juverderm voluma radiesse fill loss volume...   

                                           tokenized  
0  [severe, pain, nausea, prescribed, medication,...  
1  [severe, bleeding, large, clot, last, week, fi...  
2  [take, medicine, able, find, heartbeat, bad, p...  
3  [really, early, morning, go, take, tylenol, mi...  
4  [try, juverderm, voluma, radiesse, fill, loss,...  
[TaggedDocument(words=['severe', 'pain', 'nausea', 'prescribed', 'medication', '

In [17]:
print(tagged_data[:5])

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=10, compute_loss = True)
    print("training loss = ", model.get_latest_training_loss())
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
doc_list = 'increase b number plus alot energy lose weight'.split(' ')

inferred_vector = model.infer_vector(doc_list)
return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=5)
print(return_docs)
for rd in return_docs:
  for des in df_train[df_train['condition2'] == rd[0]]['review']:
    print(rd[0], rd[1], des)
print("done")

[TaggedDocument(words=['severe', 'pain', 'nausea', 'prescribed', 'medication', 'still', 'pain', 'nausea', 'tolerable', 'severe', 'pain', 'nausea', 'prescribed', 'medication', 'still', 'pain', 'nausea', 'tolerable'], tags=['abdominaldistension']), TaggedDocument(words=['severe', 'bleeding', 'large', 'clot', 'last', 'week', 'finally', 'saw', 'doctor', 'give', 'mg', 'day', 'day', 'stop', 'bleeding', 'day', 'ago', 'love', 'medicine', 'really', 'hungry', 'often', 'okay', 'unsure', 'happen', 'stop', 'take', 'get', 'st', 'depo', 'injection', 'middle', 'june', 'start', 'bleeding', 'middle', 'july', 'stop', 'one', 'week', 'nd', 'injection', 'one', 'week', 'prior', 'date', 'nd', 'injection', 'due', 'start', 'bleeding', 'like', 'period', 'bleeding', 'stop', 'part', 'get', 'little', 'bit', 'trace', 'nothing', 'noticeable', 'two', 'month', 'bleeding', 'worth', 'make', 'monthly', 'delight', 'end', 'use', 'depo', 'year', 'notice', 'first', 'year', 'would', 'spot', 'lightly', 'usually', 'get', 'period

training loss =  0.0
iteration 1
training loss =  0.0
iteration 2
training loss =  0.0
iteration 3
training loss =  0.0
iteration 4
training loss =  0.0
iteration 5
training loss =  0.0
iteration 6
training loss =  0.0
iteration 7
training loss =  0.0
iteration 8
training loss =  0.0
iteration 9
training loss =  0.0
[('thyroidsuppressiontest', 0.6704919338226318), ('hyperphosphatemia', 0.660398542881012), ('jointinfection', 0.6533063650131226), ('gonadotropininhibition', 0.6473581790924072), ('typhoidfeve', 0.6349539756774902)]
thyroidsuppressiontest 0.6704919338226318 gain weight 
hyperphosphatemia 0.660398542881012 work well medoes consipate good job lower phosphorus work well medoes consipate good job lower phosphorus 
jointinfection 0.6533063650131226 major gastrointestinal side effect include nausea diarrhea 
gonadotropininhibition 0.6473581790924072 severe nausea headache weight loss depression 
typhoidfeve 0.6349539756774902 good result good result 
done


<ipython-input-17-72defe17d47a>:16: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=5)


In [ ]:
while True:
    test = input("what ? = ").split(' ')
    inferred_vector = model.infer_vector(test)
    return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=5)
    print(return_docs)
    for rd in return_docs:
      for des in df_train[df_train['condition2'] == rd[0]]['review']:
        print(rd[0], rd[1], des)
    print("done")


In [5]:
!pip list

Package                            Version
---------------------------------- -------------------
-                                  autifulsoup4
-eautifulsoup4                     4.9.3
absl-py                            0.12.0
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 1.10.0
anaconda-project                   0.8.3
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.4.2
astropy                            4.0.2
astunparse                         1.6.3
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.4
Babel                              2.8.1
backcall                           0.2.0
backports.functools-lru-cache      1.6.1
backports.shutil-get-terminal-size 1.0.0
backports.tempfile           